<a href="https://colab.research.google.com/github/semishen/Chinese-NLP-Practices/blob/main/06_MultinomialNB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# load data
url = 'https://raw.githubusercontent.com/ywchiu/tibame_tm/master/data/finance_article.xlsx'
finance = pd.read_excel(url)
print(finance['tag'].value_counts())
finance.head()

Loan          100
creditcard    100
Stock         100
Name: tag, dtype: int64


,url,title,content,tag
0,https://www.ptt.cc/bbs/Loan/M.1586506724.A.914...,[問題] 新北原屋貸款,[抵押房屋資訊]\n地點：新北市永和\n房屋形態：透天五樓\n房屋種類：自有住宅\n房屋/土...,Loan
1,https://www.ptt.cc/bbs/Loan/M.1586449508.A.618...,信貸 (台積電),1. 信貸金額：80-200萬\n2. 還款期限：7年\n3. 貸款用途：投資理財\n4. ...,Loan
2,https://www.ptt.cc/bbs/Stock/M.1586573726.A.DB...,[新聞] 瑞幸咖啡遭遇做空 疫情危機下中概股受到的放大鏡式狙擊,1.原文連結：\n\n2.原文內容：\n林祖偉／BBC中文\n\n瑞幸咖啡有超過4500間分...,Stock
3,https://www.ptt.cc/bbs/Stock/M.1586582519.A.AE...,[新聞] 14檔業績四升 法人搶,1.原文連結：\n\n2.原文內容：14檔業績四升 法人搶\n2020-04-11 00:1...,Stock
4,https://www.ptt.cc/bbs/Stock/M.1586572758.A.8B...,[新聞] 突變酵素 塑膠瓶幾小時內能回收,1.原文連結：\n\n2.原文內容：\n科學家創造出一種突變細菌酵素，能在幾小時分解塑膠瓶進...,Stock


In [ ]:
# words segmentation
import jieba

corpus = []
titles = finance['title'].to_numpy()
tags = finance['tag'].to_numpy()

for _, row in finance.iterrows():
  corpus.append(' '.join(jieba.cut(row['content'])))

print(len(corpus))
corpus

300


['[ 抵押 房屋 資訊 ] \n 地點 ： 新北市 永和 \n 房屋 形態 ： 透天 五樓 \n 房屋 種類 ： 自有 住宅 \n 房屋 / 土地 所有人 ： 母親 \n 坪數 ： 45 - 50 \n [ 需求 資訊 ] \n 欲 貸金額 :   500 - 1000 萬 \n 欲貸 年期 ： 20 / 30 年 \n 欲求 利率 ： 降息 後 的 行情 ， 理財型 希望 在 1.9% 以下 ， 一般 型 1.5% 以下 ( 目前 可 做 1.58% ) \n 貸款 用途 ： 投資理 財消費 \n 希望 還款 條件 ： 優先 徵求 長 年期 理財型 房貸 利率 \n [ 貸款 人 資訊 ] \n 年齡 ： 33 \n 職業 ： 金融 業務 \n 年收入 ： 100 萬 ， 本業加 其他 收入 \n 年資 ： 5 年 \n 信用卡 ： 匯豐 、 富邦 、 台新 \n 麻煩 來 信 提供 以下 內容 ： \n 1 . 銀行 及 方案 \n 2 . 總費用 ： 包含 開辦費 、 設定費 、 手續費 等等 相關 費用 ） \n 3 . 利率 方式 \n 4 . 綁 約 時間 / 提前 償 還規定 及 利率',
 '1 .   信貸 金額 ： 80 - 200 萬 \n 2 .   還款 期限 ： 7 年 \n 3 .   貸款 用途 ： 投資理財 \n 4 .   職業 ：   資深 工程 師   ( 台積電 ) \n 5 .   年收入 ：   150 萬 \n 6 .   年資 ：   年資 8 年 \n 7 .   信用卡 :   中信 / 國泰 / 玉山 / 永豐 等   繳款 正常 ,   全額 繳清 \n 8 .   其他 貸款 ： 無 \n 9 .   所在地 ：   新竹 \n 麻煩 來 信請 回覆 以下 資訊 \n 1 . 銀行 與 方案 名稱 \n 2 . 開辦 費用 或 手續費 \n 3 . 利率 計算 方式 ： 一段 式 ， 2.1% 以下 \n 4 . 總費用 ( 設定費 、 手續費 及其 它 相關 費用 ） \n 5 . 提前 清償 違約 金 與 期限 等 相關 規定 （ 綁 約 期限 、 提前 還款 違約金 ） \n 6 . 每月 應繳 金額 及 還款 總金額',
 '1 . 原文 連結 ： \n \n 2 . 原文 內容 ： \n 林祖偉

In [ ]:
# compute TF_IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(corpus)
tfidf

<300x9613 sparse matrix of type '<class 'numpy.float64'>'
	with 26429 stored elements in Compressed Sparse Row format>

In [ ]:
# split data to train and test
train_x, test_x, train_y, test_y, train_titles, test_titles = train_test_split(tfidf, tags, titles, test_size = 0.2, random_state=1)
print(train_x.shape, test_x.shape)

(240, 9613) (60, 9613)


In [ ]:
# build MultinomialNB and validate model
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

clf = MultinomialNB(alpha=0.01)
clf.fit(train_x, train_y)
test_predict = clf.predict(test_x)

acc = accuracy_score(test_y, test_predict)
print('Accuracy: ', acc)


Accuracy:  0.9833333333333333


In [ ]:
# compute confusion_matrix
con_m = confusion_matrix(test_y, test_predict)

print('confusion matrix: \n', clf.classes_)
print(con_m)
print('===========================')

for i,(label, predict) in enumerate(zip(test_y, test_predict)):
  if label != predict:
    print('Wrong Prediction Content: ')
    print(f'test_label: {label} | predict_label: {predict}')
    print(f'title: {test_titles[i]}')
    print('content: ')
    print(finance[finance['title'] == test_titles[i]]['content'].values)
    


confusion matrix: 
 ['Loan' 'Stock' 'creditcard']
[[20  0  0]
 [ 0 19  1]
 [ 0  0 20]]
Wrong Prediction Content: 
test_label: Stock | predict_label: creditcard
title: [請益] 所得稅率基本扣除與股利所得？
content: 
['依照小弟所得稅基本抵稅額\n1.12萬個人資扣除額\n2.17萬基本生活費 \n3.扶養親屬\n4.身心障礙者\n免稅額度大概40萬+53萬以下 ，小弟年所得約90萬，但是儲蓄險到期約40萬想要存股單純吃\n股利。\n       但是怕明年股利配現金3塊左右\n股價40萬左右光寶或超豐  股利所得3萬或4萬變成要扣12啪的稅率，90萬*6啪=5.4萬，今年\n好像股利所得計算方式又改了。\n還是除息前賣出在除息當天買回那個會比較好。']
